In [1]:
from sklearn.svm import SVC
from functions import *
from sklearn.preprocessing import StandardScaler

In [2]:
stock= 'gme'
stock_df = f'{stock}_data'

In [3]:
globals()[stock_df] = data_loading_processing(stock)[2]


In [4]:
globals()[stock_df].head()

,Year,Year_Week,Begin_Price,Adj Close,Return,Label,Week_Number,mean_return,volatility
0,2017,2017-01,6.36,4.99,0.784591,NoLabel,1,-0.82525,2.466974
1,2017,2017-02,4.99,4.62,0.925852,NoLabel,2,-1.44700,3.705994
2,2017,2017-03,4.62,4.79,1.036797,NoLabel,3,0.92975,1.424463
3,2017,2017-04,4.79,4.94,1.031315,NoLabel,4,0.61380,0.731913
4,2017,2017-05,4.94,5.03,1.018219,NoLabel,5,0.35540,1.023493


In [5]:
svc_linear = SVC(kernel='linear')
svc_gaussian = SVC(kernel='rbf')
svc_poly = SVC(kernel='poly',degree=2)
models = {'svc_linear': svc_linear,
          'svc_gaussian':svc_gaussian,
          'svc_poly':svc_poly}

In [6]:
results = pd.DataFrame(columns=['TP','FP','TN','FN','accuracy','TPR','TNR'])


In [7]:
stock_2020 = globals()[stock_df].query('Year == 2020')
stock_2021 = globals()[stock_df].query('Year == 2021')
X_train = stock_2020[['mean_return','volatility']].values
y_train = stock_2020['Label'].values
X_test = stock_2021[['mean_return','volatility']].values
y_test = stock_2021['Label'].values
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)
trading_df = pd.DataFrame(columns = ['Trading','B&H'])
for model_name, m in models.items():
    m.fit(X_train,y_train)
    preds = m.predict(X_test)
    results = compile_results(results,m,preds,y_test,model_name = model_name)
    trading_df.loc[model_name,['B&H','Trading']] = \
    [buy_and_hold(globals()[stock_df],2021),
     trade_labels(globals()[stock_df],2021,
                  np.array([1 if p == 'Green' else 0 for p in preds]))]
# for model_name, m in models.items():

#     results.loc[model_name,'B&H'] = buy_and_hold(globals()[stock_df],2021)
#     results.loc[model_name,'Trading'] = trade_labels(globals()[stock_df],2021,)

combined_results = pd.concat([results,trading_df],axis=1)

combined_results

,TP,FP,TN,FN,accuracy,TPR,TNR,Trading,B&H
svc_linear,27,1,17,7,0.85,0.79,0.94,45327.32,807.43
svc_gaussian,28,1,17,6,0.87,0.82,0.94,9172.4,807.43
svc_poly,29,11,7,5,0.69,0.85,0.39,2599.65,807.43


In [8]:
globals()[f'{stock}_results'] = combined_results

### SPY

In [9]:
stock= 'spy'
stock_df = f'{stock}_data'

In [10]:
globals()[stock_df] = data_loading_processing(stock)[2]

In [11]:
globals()[stock_df].head()

,Year,Year_Week,Begin_Price,Adj Close,Return,Label,Week_Number,mean_return,volatility
0,2017,2017-01,225.04,202.38,0.899307,NoLabel,1,0.2185,0.314887
1,2017,2017-02,202.38,202.24,0.999308,NoLabel,2,-0.0136,0.275665
2,2017,2017-03,202.24,201.97,0.998665,NoLabel,3,-0.0335,0.382903
3,2017,2017-04,201.97,203.95,1.009803,NoLabel,4,0.1970,0.517220
4,2017,2017-05,203.95,204.28,1.001618,NoLabel,5,0.0332,0.463586


In [12]:
svc_linear = SVC(kernel='linear')
svc_gaussian = SVC(kernel='rbf')
svc_poly = SVC(kernel='poly',degree=2)
models = {'svc_linear': svc_linear,
          'svc_gaussian':svc_gaussian,
          'svc_poly':svc_poly}

In [13]:
results = pd.DataFrame(columns=['TP','FP','TN','FN','accuracy','TPR','TNR'])


In [14]:
stock_2020 = globals()[stock_df].query('Year == 2020')
stock_2021 = globals()[stock_df].query('Year == 2021')
X_train = stock_2020[['mean_return','volatility']].values
y_train = stock_2020['Label'].values
X_test = stock_2021[['mean_return','volatility']].values
y_test = stock_2021['Label'].values
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)
trading_df = pd.DataFrame(columns = ['Trading','B&H'])
for model_name, m in models.items():
    m.fit(X_train,y_train)
    preds = m.predict(X_test)
    results = compile_results(results,m,preds,y_test,model_name = model_name)
    trading_df.loc[model_name,['B&H','Trading']] = \
    [buy_and_hold(globals()[stock_df],2021),
     trade_labels(globals()[stock_df],2021,
                  np.array([1 if p == 'Green' else 0 for p in preds]))]
# for model_name, m in models.items():

#     results.loc[model_name,'B&H'] = buy_and_hold(globals()[stock_df],2021)
#     results.loc[model_name,'Trading'] = trade_labels(globals()[stock_df],2021,)

combined_results = pd.concat([results,trading_df],axis=1)

combined_results

,TP,FP,TN,FN,accuracy,TPR,TNR,Trading,B&H
svc_linear,19,1,30,2,0.94,0.9,0.97,164.84,127.54
svc_gaussian,21,1,30,0,0.98,1.0,0.97,162.51,127.54
svc_poly,0,0,31,21,0.60,0.0,1.00,129.05,127.54


In [15]:
globals()[f'{stock}_results'] = combined_results

In [16]:
both_stock_results = pd.concat([spy_results,gme_results],keys=['SPY','GME'],axis=0)

In [17]:
both_stock_results

TP  FP  TN  FN  accuracy   TPR   TNR   Trading     B&H
SPY svc_linear    19   1  30   2      0.94  0.90  0.97    164.84  127.54
    svc_gaussian  21   1  30   0      0.98  1.00  0.97    162.51  127.54
    svc_poly       0   0  31  21      0.60  0.00  1.00    129.05  127.54
GME svc_linear    27   1  17   7      0.85  0.79  0.94  45327.32  807.43
    svc_gaussian  28   1  17   6      0.87  0.82  0.94    9172.4  807.43
    svc_poly      29  11   7   5      0.69  0.85  0.39   2599.65  807.43